In [1]:
import numpy as np
import pandas as pd
data = pd.read_csv("amazon_baby.csv")
#pd.options.display.max_rows=10

In [2]:
data

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5
6,A Tale of Baby's Days with Peter Rabbit,"Lovely book, it's bound tightly so you may not...",4
7,"Baby Tracker&reg; - Daily Childcare Journal, S...",Perfect for new parents. We were able to keep ...,5
8,"Baby Tracker&reg; - Daily Childcare Journal, S...",A friend of mine pinned this product on Pinter...,5
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4


In [3]:
def remove_punctuation(text):
    import string
    return text.translate(str.maketrans('','',string.punctuation))

In [4]:
data['review'] = data['review'].fillna('')
data['review_clean'] = data['review'].apply(remove_punctuation)

In [13]:
#2) Extract Sentiments
#remove all 3* reviews
data = data[data['rating'] != 3]

data['sentiment'] = data['rating'].apply(lambda x : +1 if x > 3 else -1)

In [6]:
pd.options.display.max_rows=10
data['sentiment']

1         1
2         1
3         1
4         1
5         1
         ..
183526    1
183527    1
183528    1
183529    1
183530    1
Name: sentiment, Length: 166752, dtype: int64

In [7]:
#3) Split into training and test sets
#vtrain_data,test_data = data.random_split(.8, seed=0)
#train_data, test_data = np.split(data.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
msk = np.random.rand(len(data)) < 0.8
train_data = data[msk]
test_data = data[~msk]

In [8]:
#4) Build the word count vector for each review
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern = r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
test_matrix = vectorizer.fit_transform(test_data['review_clean'])

In [9]:
#5) Train a sentiment classifier with logistic regression
from sklearn.linear_model import LogisticRegression
logisticRegression = LogisticRegression()
sentiment_model = logisticRegression.fit(train_matrix, y=train_data["sentiment"])

In [10]:
# 5 c.d. + 6) Prediciting Sentiment
y = []
counter = 0
arrSize = sentiment_model.coef_.size
for i in range(arrSize):
    if sentiment_model.coef_.flat[i] >= 0:
        counter += 1
        y.append(+1)
    else:
        y.append(-1)
print("Number of positive reviews: ", counter)

Number of positive reviews:  87270


In [11]:
a = sentiment_model.predict(train_matrix)
a.mean()
sentiment_model.coef_

array([[ -6.08928279e-01,   1.03381120e-04,   4.37814302e-02, ...,
          5.18422579e-06,   9.46598551e-03,   1.03125920e-02]])

In [12]:
# 7) Probability Predictions
P = []
arrSize = sentiment_model.coef_.size
for i in range(arrSize):
    P.append(1 / (1 + np.exp(-sentiment_model.coef_.flat[i])))
P.mean()

AttributeError: 'list' object has no attribute 'mean'

In [20]:
# 8) Find the most positive (and negative) review

test_data["probability"] = sentiment_model.predict_proba(train_matrix)

top_20 = test_data.topk("probability", k=20)
top_20.print_rows(20)

bottom_20 = test_data.topk("probability", k=20, reverse = True)
bottom_20.print_rows(20)

ValueError: Length of values does not match length of index

In [23]:
# 9) Learn another classifier with fewer words
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [48]:
vectorizer_word_subset = CountVectorizer(input=significant_words)
vectorizer_word_subset

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8',
        input=['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 'work', 'product', 'money', 'would', 'return'],
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [46]:
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.fit_transform(test_data['review_clean'])

In [47]:
# 10) Train a logistic regression model on a subset of data
# Now build a logistic regression classifier with train_matrix_word_subset as features and sentiment
# as the target. Call this model simple_model. Let us inspect the weights (coefficients) of the simple_model.
# First, build a table to store (word, coefficient) pairs.
# Sort the data frame by the coefficient value in descending order.
logisticRegression_word_subset = LogisticRegression()
simple_model = logisticRegression_word_subset.fit(train_matrix_word_subset, y=train_data["sentiment"])

In [53]:
print(simple_model.coef_)
table = list(zip(significant_words,simple_model.coef_))

[[  8.63572007e-05   4.03133374e-02  -1.88283031e-01 ...,   6.39444971e-06
    1.07258262e-02   9.62739302e-03]]
[('love', array([  8.63572007e-05,   4.03133374e-02,  -1.88283031e-01, ...,
         6.39444971e-06,   1.07258262e-02,   9.62739302e-03]))]
